# 2.4 LangGraph 내장 도구(tool)를 활용해서 만드는 Agent
- LangGraph는 많은 [내장 도구들](https://python.langchain.com/docs/integrations/tools/)을 제공합니다
- 내장 도구를 활용해서 에이전트를 만드는 방법을 알아봅니다
- 커스텀 도구를 만들기 전에 내장 도구를 먼저 확인하면 개발 효율을 높일 수 있습니다

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

- 간단한 웹 검색 도구를 활용합니다 
- [Tavily](https://tavily.com/)는 매월 1000건의 무료 호출을 제공하지만 [DuckDuckGo](https://duckduckgo.com/)는 완전 무료입니다

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

- [GmailToolkit](https://python.langchain.com/docs/integrations/tools/gmail)은 구글 메일 도구를 제공합니다
- 먼저 `credentials` 설정 후에 사용할 수 있습니다

In [4]:
!pip install -q langchain-google-community google-auth-oauthlib

In [14]:
from langchain_google_community import GmailToolkit

from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="./gmail_token.json",
    scopes=["https://mail.google.com/"],
    client_sercret_file="./gmail_credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()


/var/folders/qt/w7p0s8cx51q4dh533wlpsbrm0000gn/T/ipykernel_14315/4225151943.py:10: DeprecationWarning: get_gmail_credentials is deprecated and will be removed in a future version.Use get_google_credentials instead.
  credentials = get_gmail_credentials(


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=315873217245-tcqbrbscjjd94i1ng8k2k879u6gdi25l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56597%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=nzNSTfPSBT2JLtQaqtFzoQACMrXnMl&access_type=offline


/var/folders/qt/w7p0s8cx51q4dh533wlpsbrm0000gn/T/ipykernel_14315/4225151943.py:15: DeprecationWarning: build_resource_service is deprecated and will be removed in a future version.Use build_gmail_service instead.
  api_resource = build_resource_service(credentials=credentials)


In [ ]:
gmail_tool_list

- [load_tools](https://api.python.langchain.com/en/latest/agents/langchain.agents.load_tools.load_tools.html) 함수를 통해 내장 도구를 불러올 수 있습니다

In [ ]:
from langchain.agents import load_tools

loaded_tool_list = load_tools(
    ["arxiv"],
)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'chroma_collection',
    persist_directory = './chroma_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='real_estate_tax_retriever',
    description='Contains information about 청년월세지원 2025',
)

- `ToolNode`에 도구 리스트를 전달합니다

In [ ]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply, search_tool, retriever_tool] + gmail_tool_list + loaded_tool_list
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

In [ ]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [ ]:
def agent(state: MessagesState) -> MessagesState:
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {'messages': [response]}

In [ ]:
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)

In [ ]:
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition


graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges(
    'agent',
    tools_condition
)
graph_builder.add_edge('tools', 'agent')

In [ ]:
graph= graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

query = 'seungju41625886@gmail.com으로 Attention Is All You Need 논문을 요약해서 보내주세요 '
for chunk in graph.stream({'messages': [HumanMessage(query)]}, stream_mode='values'):
    chunk['messages'][-1].pretty_print()